In [5]:
import cv2
import numpy as np


def apply_mask(face: np.array, mask: np.array) -> np.array:
    #extracts the face and mask height and width
    mask_h, mask_w, _ = mask.shape
    face_h, face_w, _ = face.shape
    #resize the mask to fit the face
    factor = min(face_h/mask_h, face_w/mask_w)
    #compute the new shape
    new_mask_w = int(factor * mask_w)
    new_mask_h = int(factor * mask_h)
    new_mask_shape = (new_mask_w, new_mask_h)
    #adds the resized dog mask to the new face shape
    resized_mask = cv2.resize(mask,new_mask_shape)
    #make a copy of the image
    face_with_mask = face.copy()
    #nsure it is black and white
    non_white_pixels = (resized_mask < 250).all(axis = 2)
    #center the dog image to the face
    off_h = int((face_h - new_mask_h)/2)
    off_w = int((face_w - new_mask_w)/2)
    #copy non_white pixels from dog image to child's image
    face_with_mask[off_h: off_h+new_mask_h, off_w: off_w+new_mask_w][non_white_pixels] = \
        resized_mask[non_white_pixels]
    #return the results
    return face_with_mask


def main():
    #initialize the videocapture object
    cap = cv2.VideoCapture(0)
    #loads the dog mask
    mask = cv2.imread("/home/queen/Desktop/python/tensorflow-prac/Dog_filter/assets/dog.png")
    #initializes the face classifier
    cascade = cv2.CascadeClassifier("/home/queen/Desktop/python/tensorflow-prac/Dog_filter/assets/haarcascade_frontalface_default.xml")
    while True:
        #reads image from the camera
        ret, frame = cap.read()
        #extracts image height and width
        frame_h, frame_w, _ = frame.shape
        #converts the image to black and white
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blackwhite = cv2.equalizeHist(gray)
        #detects all faces in the image
        rects = cascade.detectMultiScale(
            blackwhite, scaleFactor = 1.3, minNeighbors = 4, minSize = (30,30),
        flags = cv2.CASCADE_SCALE_IMAGE)
        #crops the frame
        for x,y,w,h in rects:
            y0, y1 = int(y - 0.25*h), int(y + 0.75*h)
            x0, x1 = x, x + w
        #check incase the detected face is too close to the edge
            if x0 < 0 or y0 < 0 or x1 > frame_w or y1 > frame_h:
                continue
        #apply the mask
            frame[y0: y1, x0: x1] = apply_mask(frame[y0: y1, x0: x1], mask)
        #display the annotated image back to the user's screen
        cv2.imshow('frame', frame)
        #checks keyboard input from the user
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()
if __name__=='__main__':
    main()